# Import the required packages

In [1]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import joblib

# Read the data file

In [2]:
df = pd.read_csv("Amazon2024.csv", low_memory=False)
print(df.shape)
df.head()

(196798, 13)


,AMZN_YR_WK,Department,VendorName,Purchase Order,DC,Original Delivery Date,Actual Delivery Date,Gate Time,Ordercases,Rcvd Cases,Non Compliant Qty,Compliant %,Status
0,11828,85,KODAK ALARIS OPERATIONS CANADA,6250080999,6002,8/6/2024,7/9/2024,8:56:04 AM,1719,0,NaN,NaN,NaN
1,11828,85,KODAK ALARIS OPERATIONS CANADA,6350090995,6063,8/6/2024,7/11/2024,7:42:51 PM,1273,0,NaN,NaN,NaN
2,11828,85,KODAK ALARIS OPERATIONS CANADA,6450090985,6093,8/6/2024,7/12/2024,10:55:39 AM,897,0,NaN,NaN,NaN
3,11828,72,APPLE CANADA INC,8450361239,6063,8/9/2024,7/27/2024,7:02:56 AM,772,0,NaN,NaN,NaN
4,11828,98,TREASURE MILLS INC.,3300401778,6098,8/5/2024,7/29/2024,1:19:54 PM,572,0,NaN,NaN,NaN


In [3]:
df.columns = df.columns.str.strip()

for col in ["Ordercases", "Rcvd Cases"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Non-compliant qty

In [4]:
# Common operational definition: ordered - received (floor at 0)
df["Non Compliant Qty_calc"] = (df["Ordercases"] - df["Rcvd Cases"]).clip(lower=0)

# If the dataset already has "Non Compliant Qty" but missing values, fill them using our calc
if "Non Compliant Qty" in df.columns:
    df["Non Compliant Qty"] = pd.to_numeric(df["Non Compliant Qty"], errors="coerce")
    df["Non Compliant Qty"] = df["Non Compliant Qty"].fillna(df["Non Compliant Qty_calc"])
else:
    df["Non Compliant Qty"] = df["Non Compliant Qty_calc"]

print(df[["Ordercases", "Rcvd Cases", "Non Compliant Qty_calc"]].head(10))

   Ordercases  Rcvd Cases  Non Compliant Qty_calc
0        1719           0                    1719
1        1273           0                    1273
2         897           0                     897
3         772           0                     772
4         572           0                     572
5        3312           0                    3312
6        4906           0                    4906
7        1595           0                    1595
8          76           0                      76
9          19           0                      19


# Compliant percentage per order

In [5]:
# Compliant % = (Ordercases - NonCompliant) / Ordercases
# Handle divide-by-zero / missing Ordercases safely
df["Compliant %_calc"] = np.where(
    df["Ordercases"] > 0,
    (df["Ordercases"] - df["Non Compliant Qty"]) / df["Ordercases"],
    np.nan
)

# If the dataset already has "Compliant  %" but missing, fill it
if "Compliant  %" in df.columns:
    df["Compliant  %"] = pd.to_numeric(df["Compliant  %"], errors="coerce")
    df["Compliant  %"] = df["Compliant  %"].fillna(df["Compliant %_calc"])
else:
    df["Compliant  %"] = df["Compliant %_calc"]

print(df[["Ordercases", "Non Compliant Qty", "Compliant %_calc"]].head(10))

   Ordercases  Non Compliant Qty  Compliant %_calc
0        1719             1719.0               0.0
1        1273             1273.0               0.0
2         897              897.0               0.0
3         772              772.0               0.0
4         572              572.0               0.0
5        3312             3312.0               0.0
6        4906             4906.0               0.0
7        1595             1595.0               0.0
8          76               76.0               0.0
9          19               19.0               0.0


# Overall status (0/1)

In [6]:
# Simple rule: fully compliant => 1, otherwise 0
df["Status_calc"] = np.where(df["Compliant  %"] == 1, 1, 0)

# If "Status" exists but missing, fill it; otherwise create it
if "Status" in df.columns:
    df["Status"] = pd.to_numeric(df["Status"], errors="coerce")
    df["Status"] = df["Status"].fillna(df["Status_calc"])
else:
    df["Status"] = df["Status_calc"]

print(df[["Status_calc"]].head(10))

   Status_calc
0            0
1            0
2            0
3            0
4            0
5            0
6            0
7            0
8            0
9            0


# Create a useful feature = delivery delay (days)

In [7]:
# (not required by the To Do list, but helps NN)
df["Original Delivery Date"] = pd.to_datetime(df["Original Delivery Date"], errors="coerce")
df["Actual Delivery Date"] = pd.to_datetime(df["Actual Delivery Date"], errors="coerce")
df["DeliveryDelayDays"] = (df["Actual Delivery Date"] - df["Original Delivery Date"]).dt.days

df[["Ordercases","Rcvd Cases","Non Compliant Qty","Compliant  %","Status","DeliveryDelayDays"]].head()

,Ordercases,Rcvd Cases,Non Compliant Qty,Compliant %,Status,DeliveryDelayDays
0,1719,0,1719.0,0.0,0.0,-28
1,1273,0,1273.0,0.0,0.0,-26
2,897,0,897.0,0.0,0.0,-25
3,772,0,772.0,0.0,0.0,-13
4,572,0,572.0,0.0,0.0,-7


# Prepare predictors (X) and outcome (y) like the example

In [8]:
# Choose predictors (keep it reasonable; VendorName can be huge -> optional)
candidate_cols = [
    "Department", "DC",
    "Ordercases", "Rcvd Cases",
    "Non Compliant Qty", "Compliant  %",   # Note: using these makes prediction easier; ok for class exercise
    "DeliveryDelayDays"
]

# Keep only columns that exist
candidate_cols = [c for c in candidate_cols if c in df.columns]

# Drop rows with missing y or key predictors
model_df = df[candidate_cols + ["Status"]].dropna()

# Convert categorical columns to dummy variables (same idea as get_dummies in example)
processed = pd.get_dummies(model_df, columns=[c for c in ["Department","DC"] if c in model_df.columns], drop_first=True)

outcome = "Status"
predictors = [c for c in processed.columns if c != outcome]

X = processed[predictors]
y = processed[outcome].astype(int)

# Partition data

In [9]:
train_X, valid_X, train_y, valid_y = train_test_split(
    X, y, test_size=0.4, random_state=1, stratify=y
)

#  Develop the NN model (MLPClassifier)

In [49]:
#    Example uses: hidden_layer_sizes=(2), activation='logistic', solver='lbfgs'
#    We'll also scale numeric features via Pipeline (recommended for MLP).
# -----------------------------
nn_model = Pipeline(steps=[
    ("scaler", StandardScaler(with_mean=False)),  # with_mean=False works well with sparse/dummy matrices
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(2,),
        activation="logistic",
        solver="lbfgs",
        random_state=1,
        max_iter=300
    ))
])

nn_model.fit(train_X, train_y)

Pipeline(steps=[('scaler', StandardScaler(with_mean=False)),
                ('mlp',
                 MLPClassifier(activation='logistic', hidden_layer_sizes=(2,),
                               max_iter=300, random_state=1, solver='lbfgs'))])

# Model performance (train + validation)

In [50]:
train_pred = nn_model.predict(train_X)
valid_pred = nn_model.predict(valid_X)

print("Train Accuracy:", accuracy_score(train_y, train_pred))
print("Valid Accuracy:", accuracy_score(valid_y, valid_pred))

print("\nTrain Confusion Matrix:\n", confusion_matrix(train_y, train_pred))
print("\nValid Confusion Matrix:\n", confusion_matrix(valid_y, valid_pred))

print("\nValidation Classification Report:\n", classification_report(valid_y, valid_pred))


Train Accuracy: 0.9946137299073494
Valid Accuracy: 0.9944740151678756

Train Confusion Matrix:
 [[72648   626]
 [   10 44794]]

Valid Confusion Matrix:
 [[48423   426]
 [    9 29861]]

Validation Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     48849
           1       0.99      1.00      0.99     29870

    accuracy                           0.99     78719
   macro avg       0.99      1.00      0.99     78719
weighted avg       0.99      0.99      0.99     78719

